<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/draft/football_live_validation_total_heft_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[CatBoost - An In-Depth Guide [Python API]](https://coderzcolumn.com/tutorials/machine-learning/catboost-an-in-depth-guide-python#9)<br>
[Catboost](https://catboost.ai/en/docs/concepts/python-reference_pool)<br>
[Cross-Validation Techniques](https://medium.com/geekculture/cross-validation-techniques-33d389897878)

### Project config

In [1]:
try:
    import neptune.new as neptune
except:
    !pip install neptune-client >> None
    import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw
     

In [2]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Installations

In [3]:
!pip install catboost >> None

### Downloads

In [4]:
data_version = 'football_live_npz_230131/'
project = neptune.init_project(
    project="scomesse/football", 
    api_token = api_key
    )
project[data_version + 'dataset'].download('./dataset.npz')
project[data_version + 'description'].download('./save_discription.txt')
project[data_version + 'additional_data'].download('./additional_data.npz')
project[data_version + 'time'].download('./time.csv')
params = project[data_version + 'params'].fetch()
project.stop()

https://app.neptune.ai/scomesse/football/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api/project#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/metadata


### Imports

In [3]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

1.3.5
1.21.6


In [4]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [5]:
from catboost import CatBoost
from catboost import utils
from catboost import CatBoostClassifier, CatBoostRegressor
from catboost import Pool, cv
from catboost.utils import eval_metric
np.random.seed(147)

In [6]:
from tqdm import tqdm
from scipy.stats import poisson
import os, psutil

In [7]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.metrics import multilabel_confusion_matrix

### Code

#### Functions

In [10]:
def calculate_multiclass(probability_2x:np.array, line_2x:np.array):
    '''
    input : 0 - over, 1 - under
    output: 0 - over, 1 - under
    '''
    probline = probability_2x * line_2x
    best_odd_result = np.argmax(probline, axis = 1)
    best_odd_float = np.take_along_axis(
        probline,
        best_odd_result.reshape(-1, 1),
        axis = 1
    )
    return {
        'argmax':best_odd_result,
        'float':best_odd_float
    }

In [9]:
def get_profit_curve(prob_dict, final_goal_sum, Line_production, bet_type = 'fixed'):
    '''
    Line_production -> line['total', 'over', 'under']
    final_goal_sumv -> data_npz['result1'] + data_npz['result2']
    '''
    #fixed, divk, divk-1
    #simple, complex
    # считаем прдикты по исходу и вероятности умноженной на кэф (over, under)
    preds_int, preds_float = calculate_multiclass(prob_dict, Line_production[:,1:3]).values()
    preds_float = preds_float[:,0] 
    preds_vec = preds_int * (2) - 1 # -1:over, 1:under
    # вектор результатов
    res_vec = (final_goal_sum > Line_production[:, 0]) * (-1) + \
             (final_goal_sum < Line_production[:, 0]) * (1)
    win_vec, lose_vec = (preds_vec * res_vec) == 1, (preds_vec * res_vec) == -1
    line_vec = np.take_along_axis(Line_production[:,1:3], preds_int.reshape(-1, 1), axis = 1)[:, 0]
    threshold = []
    profit = []
    bet_qty_list = []
    for th in np.linspace(0.1,0.2,1001):
        threshold.append(th)
        vector_th = (preds_float / 10) > th
        bet_qty_list.append(vector_th.sum())
        if bet_type == 'divk':
            #                       размер ставки                                   кэф
            profit.append(np.sum((1/line_vec[win_vec & vector_th].astype(np.float32)) * \
                                (line_vec[win_vec & vector_th] - 1).astype(np.float32)) -\
                                np.sum((1/line_vec[lose_vec & vector_th].astype(np.float32))))
        else:
            profit.append(np.sum((line_vec[win_vec & vector_th] - 1).astype(np.float32)) - np.sum(lose_vec & vector_th))

    return threshold, profit, bet_qty_list

In [11]:
def get_profit_validation(prob_dict, final_goal_sum, Line_production,  model_name):
    '''
    y_true - numpy вектор, shape:(x,) истинные значения в формате 0|1
    x_predicted - numpy вектор, shape:(x,) предикт (probability (float)) в формате 0.
    Line_production - numpy вектор, shape:(x,) вектор коэфициентов в формате float 1.
    '''
    scatters_dicts = dict(
        scatter1 = dict(x = [], y = [], name = '', fill = 'tozeroy', yaxis = '', xaxis = ''),
        scatter2 = dict(x = [], y = [], name = '', line = dict(color='rgb(33,113,181)', dash='dash'), yaxis = '', xaxis = ''),
        scatter3 = dict(x = [], y = [], name = '', line = dict(color='rgb(107,174,214)', dash='dash'), yaxis = '', xaxis = '')
                    )
    qty_color = 'blue'
    prof_qty_color = 'rgb(8,48,107)'
    bet_type_list = ['fixed', 'divk']
    bet_size_list = ['1', '1/K']
    strategy_name_list = ['threshold', 'pred*k']
    #domain_list = [[0.55, 1], [0., 0.5]]
    layout_dict = {}
    data_list = []
    title_text = f'Profit & bet qty for validation model in neptune.ai: {model_name}'
    for cnt_bet, bet_type in enumerate(bet_type_list):
        y_anchor = str((cnt_bet*3) + 1)
        threshold, profit, bet_qty_list = get_profit_curve(
            prob_dict, final_goal_sum, Line_production, bet_type = bet_type
            )
        xaxis_num = str((cnt_bet + 1))
        layout_dict.update({
                'xaxis' + xaxis_num:{
                    'domain':[0.5 * cnt_bet, 0.5 * cnt_bet + 0.5],
                    'title':f'bet={bet_size_list[cnt_bet]}',
                    'anchor':'y' + y_anchor}
                            })
        for cnt_scatter in range(1, 4):
            yaxis_num = str((cnt_bet*3) + cnt_scatter)
            scatter_num = 'scatter' + str(cnt_scatter)
            scatters_dicts[scatter_num]['x'] = threshold
            if cnt_scatter == 1:
                scatters_dicts[scatter_num]['y'] = profit
                scatters_dicts[scatter_num]['name'] = 'profit_' +xaxis_num
                scatters_dicts[scatter_num]['xaxis'] = 'x' + xaxis_num
                scatters_dicts[scatter_num]['yaxis'] = 'y' + yaxis_num
                layout_dict.update({
                    'yaxis' + yaxis_num:{
                        #'domain':domain_list[cnt_str],
                        'title':'', #'Profit',
                        'range':[-10,int(max(profit) * 1.1)],
                        'anchor':'x' + xaxis_num
                }})
            elif cnt_scatter == 2:
                scatters_dicts[scatter_num]['y'] = np.array(profit) / np.array(bet_qty_list)
                scatters_dicts[scatter_num]['name'] = 'profit_' + xaxis_num
                scatters_dicts[scatter_num]['xaxis'] = 'x' + xaxis_num
                scatters_dicts[scatter_num]['yaxis'] = 'y' + yaxis_num
                layout_dict.update({
                    'yaxis' + yaxis_num:{
                        #'domain':domain_list[cnt_str],
                        'title':'', 'zeroline':True,
                        'side':'right', 'anchor':'x' + xaxis_num,
                        'overlaying':'y' + y_anchor}})
            elif cnt_scatter == 3:
                scatters_dicts[scatter_num]['y'] = np.array(bet_qty_list) / 1000
                scatters_dicts[scatter_num]['name'] = 'profit_' + xaxis_num
                scatters_dicts[scatter_num]['xaxis'] = 'x' + xaxis_num
                scatters_dicts[scatter_num]['yaxis'] = 'y' + yaxis_num
                layout_dict.update({
                    'yaxis' + yaxis_num:{
                        #'domain':domain_list[cnt_str],
                        'visible':False, 'showgrid':True,
                        'side':'right', 'anchor':'x' + xaxis_num,
                        'overlaying':'y' + y_anchor}})
            data_list += [go.Scatter(**scatters_dicts['scatter' + str(cnt_scatter)])]
    layout_dict.update({
        'width':1400,
        'height':800,
        'title_x':0.5,
        'title_text':title_text,
        'paper_bgcolor':'rgb(229, 237, 247)',
        'plot_bgcolor':'rgb(229, 237, 247)',
        'showlegend':False 
                        })
    layout = go.Layout(**layout_dict)
    return go.Figure(data=data_list, layout=layout)
     

In [30]:
def plot_equity(prob_dict, final_goal_sum, Line_production, th, model_name, x_date = np.empty(()), bet_type = 'fixed', data_split = 'validation'):
    #fixed, divk, divk-1
    #simple, complex
    preds_int, preds_float = calculate_multiclass(prob_dict, Line_production[:,1:3]).values()
    preds_float = preds_float[:,0] 
    preds_vec = preds_int * (2) - 1 # -1:over, 1:under
    res_vec = (final_goal_sum > Line_production[:, 0]) * (-1) + \
            (final_goal_sum < Line_production[:, 0]) * (1)
    win_vec, lose_vec = (preds_vec * res_vec) == 1, (preds_vec * res_vec) == -1
    line_vec = np.take_along_axis(Line_production[:,1:3], preds_int.reshape(-1, 1), axis = 1)[:, 0]
    vector_th = (preds_float / 10) > th
    if bet_type == 'divk':
        bet_sum = np.sum(1/line_vec[win_vec & vector_th].astype(np.float32))
        mean_bet = np.mean(1/line_vec[win_vec & vector_th].astype(np.float32))
        #                       размер ставки                                   кэф
        y = np.cumsum(((line_vec - 1) / line_vec) * (win_vec & vector_th).astype(np.float32)) -\
                            np.cumsum((1/line_vec) * (lose_vec & vector_th).astype(np.float32))
    else:
        mean_bet = 1
        bet_sum = np.sum(win_vec & vector_th)
        y = np.cumsum((line_vec - 1) * (win_vec & vector_th).astype(np.float32)) - np.cumsum(lose_vec & vector_th)
    bet_qty = vector_th.sum()
    title_text = f'Equity Curve {data_split} | threshold={th} | bet_type:{bet_type}<br>' + \
                f'bet_mean: {np.round(mean_bet, 2)} | ROI: {np.round(y[-1] * 100 / bet_sum, 4)}%' + \
                f' | Bet quantity: {bet_qty}<br>' + f'Model in neptune.ai: FOOT-{model_name}'
    
    layout_dict = {}
    if x_date.shape:
        trace_equity = go.Scatter(y=y[(win_vec|lose_vec)&vector_th], x= x_date[(win_vec|lose_vec)&vector_th], fill='tozeroy', xaxis = 'x1') #'toself'
        layout_dict.update({'xaxis1':{'showgrid':True, 'ticklabelmode':'period', 'tickformat':'%d\n%b\n%Y'}})
        data_list = [trace_equity]
    else:
        trace_equity = go.Scatter(y = y, fill='tozeroy', xaxis = 'x1', yaxis = 'y1') #'toself'
        layout_dict.update({'xaxis1':{'anchor':'y1'}, 'yaxis1':{'domain':[0.4, 1]}})
        max_profit = np.maximum.accumulate(y)
        trace_drawdown =  go.Scatter(y = (y - max_profit) / mean_bet, fill='tozeroy', xaxis = 'x2', yaxis = 'y2')
        layout_dict.update({'xaxis2':{'anchor':'y2'}, 'yaxis2':{'domain':[0., 0.35], 'title':'drawdown inmean(bet)'}})
        data_list = [trace_equity, trace_drawdown]

    layout_dict.update({
    'width':1400,
    'height':800,
    'title_x':0.5,
    'title_text':title_text,
    'paper_bgcolor':'rgb(229, 237, 247)',
    'plot_bgcolor':'rgb(229, 237, 247)',
    'showlegend':False 
                    })
     #, trace_time_dd]
    layout = go.Layout(**layout_dict)
    return go.Figure(data=data_list, layout=layout)

#### Load Data

In [12]:
dataset_name = './dataset.npz'
data_npz = np.load(dataset_name)
X_train, X_test, X_holdout = data_npz['X_train'], data_npz['X_test'], data_npz['X_holdout']

In [13]:
scores_list = ['score1_train', 'score2_train', 'result1_train', 'result2_train', 'score1_test', 
'score2_test', 'result1_test', 'result2_test', 'score1_holdout', 'score2_holdout',
'result1_holdout', 'result2_holdout']
scores_dict = {}
for item in scores_list:
    scores, data_type, team = item.split('_')[0][:-1], item.split('_')[1], item.split('_')[0][-1]
    if scores in scores_dict:
        if data_type in scores_dict[scores]:
            scores_dict[scores][data_type][team] = data_npz[item]
        else:
            scores_dict[scores].update({data_type:{team:data_npz[item]}})
    else:
        scores_dict[scores] = {data_type:{team:data_npz[item]}}

y_train, line_train, active_train =  data_npz['y_train_multi'], data_npz['K_train'][:,1:], data_npz['K_train'][:,0] == 1
y_train_regr1 = data_npz['y_train_regression1']
y_train_regr2 = data_npz['y_train_regression2']
y_test, line_test, active_test  =  data_npz['y_test_multi'], data_npz['K_test'][:,1:], data_npz['K_test'][:,0] == 1
y_test_regr1 = data_npz['y_test_regression1']
y_test_regr2 = data_npz['y_test_regression2']
y_holdout, line_holdout, active_holdout = data_npz['y_holdout_multi'], data_npz['K_holdout'][:,1:], data_npz['K_holdout'][:,0] == 1
y_holdout_regr1 = data_npz['y_holdout_regression1']
y_holdout_regr2 = data_npz['y_holdout_regression2']
zero_train, zero_test, zero_holdout = line_train > 1, line_test > 1, line_holdout > 1

model_head = 'FOOT-LIVEBST1'
model_head2 = 'FOOT-LIVEBST2'

#### Download models and restore predicts

In [14]:
# Модель для 1-ой команды
model_num = 3 # Указываем номер модели
neptune_model = model_head
neptune_model_version = neptune_model + f'-{model_num}'
model_version_params = dict(
    project = 'scomesse/football',
    model = neptune_model,
    api_token = api_key,
    with_id = neptune_model_version
)
PATH_TO_MODEL = './booster.model'
model_version = neptune.init_model_version(**model_version_params)
model_version['model'].download(PATH_TO_MODEL)
params1 = model_version['team_parameters'].fetch()
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-3
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-3/metadata


In [15]:
# Модель для 2-ой команды
model_num_2 = 4 # Указываем номер модели
neptune_model_2 = model_head2
neptune_model_version_2 = neptune_model_2 + f'-{model_num_2}'
model_version_params2 = dict(
    project = 'scomesse/football',
    model = neptune_model_2,
    api_token = api_key,
    with_id = neptune_model_version_2
)
PATH_TO_MODEL = './booster2.model'
model_version2 = neptune.init_model_version(**model_version_params2)
model_version2['model'].download(PATH_TO_MODEL)
params2 = model_version2['team_parameters'].fetch()
model_version2.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST2/v/FOOT-LIVEBST2-4
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST2/v/FOOT-LIVEBST2-4/metadata


In [16]:
preds_dict = {}
booster = CatBoost()
booster.load_model('./booster.model')
preds_dict['train'] = {1:booster.predict(X_train) * 21.0}
preds_dict['test'] = {1:booster.predict(X_test) * 21.0}
preds_dict['holdout'] = {1:booster.predict(X_holdout) * 21.0}

booster2 = CatBoost()
booster2.load_model('./booster2.model')
preds_dict['train'].update({2:booster2.predict(X_train) * 21.0})
preds_dict['test'].update({2:booster2.predict(X_test) * 21.0})
preds_dict['holdout'].update({2:booster2.predict(X_holdout) * 21.0})
#---------------
process = psutil.Process(os.getpid())
print('Mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------

Mem usage:  3.5 GiB


#### Calculate probabilities

In [17]:
active_rows_dict ={}
active_rows_dict['train'] = (active_train & (line_train[:,6:9].sum(axis = 1) > 0))
active_rows_dict['test'] = (active_test & (line_test[:,6:9].sum(axis = 1) > 0))
active_rows_dict['holdout'] = (active_holdout & (line_holdout[:,6:9].sum(axis = 1) > 0))

In [18]:
# Считаем Пуассона
poisson_dict = {}
for name in ['train', 'test', 'holdout']:
    poisson_dict[name] = {}
    poisson_dict[name][1] = {}
    poisson_dict[name][2] = {}
    for goal in range(7):
        poisson_dict[name][1][goal] = poisson.pmf(goal, preds_dict[name][1][active_rows_dict[name]])
        poisson_dict[name][2][goal] = poisson.pmf(goal, preds_dict[name][2][active_rows_dict[name]])

In [19]:
# Считаем вероятности суммы забитых мячей
total_matrix_dict = {}
for name in ['train', 'test', 'holdout']:
    total_matrix_dict[name] = np.zeros((np.sum(active_rows_dict[name]), 13))
    for goal1 in range(7):
        for goal2 in range(7):
            total_matrix_dict[name][:, goal1 + goal2] = total_matrix_dict[name][:, goal1 + goal2] + \
            poisson_dict[name][1][goal2] * poisson_dict[name][2][goal1]

In [20]:
# Считаем вероятности забить не менее определенного количества мячей
over_matrix_dict = {}
for name in ['train', 'test', 'holdout']:
    over_matrix_dict[name] = np.hstack((
        np.flip(np.cumsum(np.flip(total_matrix_dict[name], axis = 1), axis = 1), axis = 1), 
        np.zeros(np.sum(active_rows_dict[name])).reshape(-1,1)
                                    ))[:, 1:]

In [21]:
# Считаем вероятности забить не более определенного количества
under_matrix_dict = {}
for name in ['train', 'test', 'holdout']:
    under_matrix_dict[name] = np.hstack((
            np.zeros(np.sum(active_rows_dict[name])).reshape(-1,1), 
            np.cumsum(total_matrix_dict[name], axis = 1)
                                            ))[:, :-1]

In [22]:
# Считаем текущей стэйт суммы забитых мячей
total_state_dict = {}
total_state_dict['train'] = line_train[:,6][active_rows_dict['train']] - \
                            scores_dict['score']['train']['1'][active_rows_dict['train']] - \
                            scores_dict['score']['train']['2'][active_rows_dict['train']]
total_state_dict['train'][total_state_dict['train'] < 0] = 0
total_state_dict['test'] =  line_test[:,6][active_rows_dict['test']] - \
                            scores_dict['score']['test']['1'][active_rows_dict['test']] - \
                            scores_dict['score']['test']['2'][active_rows_dict['test']]
total_state_dict['test'][total_state_dict['test'] < 0] = 0
total_state_dict['holdout'] = line_holdout[:,6][active_rows_dict['holdout']] - \
                            scores_dict['score']['holdout']['1'][active_rows_dict['holdout']] - \
                            scores_dict['score']['holdout']['2'][active_rows_dict['holdout']]
total_state_dict['holdout'][total_state_dict['holdout'] < 0] = 0

In [23]:
prob_dict = {}
for name in ['train', 'test', 'holdout']:
    prob_dict[name] = np.hstack((
        np.take_along_axis(over_matrix_dict[name], total_state_dict[name].reshape(-1, 1).astype(np.int8), axis = 1),
        np.take_along_axis(under_matrix_dict[name], (total_state_dict[name].reshape(-1, 1) + 0.5).astype(np.int8), axis = 1),
    ))

In [24]:
4753384 - 13258614
5203349 - 15038087
5264680 - 15038063
5795894 - 15095747

-9299853

#### Plot TEST Profit validation

In [25]:
final_goal_sum = data_npz['result1_test'][active_rows_dict['test']] + data_npz['result2_test'][active_rows_dict['test']]

In [ ]:
fig = get_profit_validation(prob_dict['test'], final_goal_sum, line_test[active_rows_dict['test'], 6:9],  'test: ' + neptune_model + f'-{model_num}')
fig.show()

In [106]:
#fig.write_html(f'test: {neptune_model}-{model_num}_profit_report.html') #neptune_model + f'-{model_num}'
model_version = neptune.init_model_version(**model_version_params)
model_version[f'total_profit_validation_test'].upload(neptune.types.File.as_html(fig))
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-3
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-3/metadata


In [ ]:
threshold = 0.1087
bet_type = 'divk'
fig = plot_equity(
    prob_dict['test'], 
    final_goal_sum, 
    line_test[active_rows_dict['test'], 6:9], 
    threshold, 
    'test: ' + neptune_model + f'-{model_num}', 
    bet_type = bet_type, 
    data_split = 'test'
    )
fig.show()

In [114]:
model_version = neptune.init_model_version(**model_version_params)
model_version[f'total_equity_test_{bet_type}_th_{threshold}'].upload(neptune.types.File.as_html(fig))
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-3
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-3/metadata


In [28]:
time_df = pd.read_csv('./time.csv', index_col = 'Id', parse_dates = ['StatTime'])
add_data_name = './additional_data.npz'
add_data = np.load(add_data_name)
id_test = add_data['id_test']
time_df = time_df[time_df.index.isin(id_test)]
time_dict = time_df.to_dict()
timeline_test_array = np.sort([np.datetime64(time_dict['StatTime'][id]) for id in id_test])
#---------------
process = psutil.Process(os.getpid())
print('Mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------

Mem usage:  5.49 GiB


In [ ]:
threshold = 0.1087
bet_type = 'divk'
fig = plot_equity(
    prob_dict['test'],
    final_goal_sum, 
    line_test[active_rows_dict['test'], 6:9], 
    threshold, 
    'test: ' + neptune_model + f'-{model_num}',
    x_date = timeline_test_array[active_rows_dict['test']],
    bet_type = bet_type, 
    data_split = 'test'
    )
fig.show()

In [167]:
model_version = neptune.init_model_version(**model_version_params)
model_version[f'timeline_total_equity_test_{bet_type}_th_{threshold}'].upload(neptune.types.File.as_html(fig))
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-3
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-3/metadata


#### Plot HOLDOUT Profit validation

In [32]:
final_goal_sum = data_npz['result1_holdout'][active_rows_dict['holdout']] + data_npz['result2_holdout'][active_rows_dict['holdout']]

In [ ]:
fig = get_profit_validation(prob_dict['holdout'], final_goal_sum, line_holdout[active_rows_dict['holdout'], 6:9],  'holdout: ' + neptune_model + f'-{model_num}')
fig.show()